In [1]:
!pip install gdown


In [2]:
import os

# Tải toàn bộ folder
!gdown --folder https://drive.google.com/drive/folders/14x4CCby6F3H4KybNpBp3pmBeYY3FXDjm -O /content/

# Giải nén tất cả file zip (nếu có)
for file in os.listdir('/content/'):
    if file.endswith(".zip"):
        os.system(f"unzip -o /content/{file} -d /content/data")


Retrieving folder contents
Processing file 1ZUCuYDOe4VVbZvNVZovpquaRQqqJQ639 img_cls_scenes_classification.zip
Processing file 1fnJMMw0LvDgl-GS4FTou5qAgLxOE2KQ0 img_cls_weather_dataset.zip
Retrieving folder contents completed
Building directory structure
Building directory structure completed
Downloading...
From (original): https://drive.google.com/uc?id=1ZUCuYDOe4VVbZvNVZovpquaRQqqJQ639
From (redirected): https://drive.google.com/uc?id=1ZUCuYDOe4VVbZvNVZovpquaRQqqJQ639&confirm=t&uuid=0da90c21-f02b-4564-b60f-5cf33fb10270
To: /content/Data/img_cls_scenes_classification.zip
100% 255M/255M [00:02<00:00, 105MB/s] 
Downloading...
From (original): https://drive.google.com/uc?id=1fnJMMw0LvDgl-GS4FTou5qAgLxOE2KQ0
From (redirected): https://drive.google.com/uc?id=1fnJMMw0LvDgl-GS4FTou5qAgLxOE2KQ0&confirm=t&uuid=a0cd3133-fd55-46d5-9f80-ad727b8fc06c
To: /content/Data/img_cls_weather_dataset.zip
100% 613M/613M [00:05<00:00, 116MB/s]
Download completed


In [3]:
import os

# Đảm bảo đường dẫn tồn tại
zip_path = "/content/Data/img_cls_scenes_classification.zip"
extract_path = "/content/Data/scenes_classification"

# Tạo thư mục giải nén (nếu chưa có)
os.makedirs(extract_path, exist_ok=True)

# Giải nén file
!unzip -o "{zip_path}" -d "{extract_path}"


Streaming output truncated to the last 5000 lines.
  inflating: /content/Data/scenes_classification/scenes_classification/train/mountain/16162.jpg  
  inflating: /content/Data/scenes_classification/scenes_classification/train/mountain/16173.jpg  
  inflating: /content/Data/scenes_classification/scenes_classification/train/mountain/16225.jpg  
  inflating: /content/Data/scenes_classification/scenes_classification/train/mountain/16322.jpg  
  inflating: /content/Data/scenes_classification/scenes_classification/train/mountain/16341.jpg  
  inflating: /content/Data/scenes_classification/scenes_classification/train/mountain/16350.jpg  
  inflating: /content/Data/scenes_classification/scenes_classification/train/mountain/16374.jpg  
  inflating: /content/Data/scenes_classification/scenes_classification/train/mountain/16375.jpg  
  inflating: /content/Data/scenes_classification/scenes_classification/train/mountain/16417.jpg  
  inflating: /content/Data/scenes_classification/scenes_classificat

In [4]:
import os

# Đường dẫn đến file zip và thư mục đích
zip_path = "/content/Data/img_cls_weather_dataset.zip"
extract_path = "/content/Data/weather_dataset"

# Tạo thư mục giải nén (nếu chưa có)
os.makedirs(extract_path, exist_ok=True)

# Giải nén file zip
!unzip -o "{zip_path}" -d "{extract_path}"


Streaming output truncated to the last 5000 lines.
  inflating: /content/Data/weather_dataset/weather-dataset/dataset/frost/3922.jpg  
  inflating: /content/Data/weather_dataset/weather-dataset/dataset/frost/3923.jpg  
  inflating: /content/Data/weather_dataset/weather-dataset/dataset/frost/3924.jpg  
  inflating: /content/Data/weather_dataset/weather-dataset/dataset/frost/3925.jpg  
  inflating: /content/Data/weather_dataset/weather-dataset/dataset/frost/3926.jpg  
  inflating: /content/Data/weather_dataset/weather-dataset/dataset/frost/3927.jpg  
  inflating: /content/Data/weather_dataset/weather-dataset/dataset/frost/3928.jpg  
  inflating: /content/Data/weather_dataset/weather-dataset/dataset/frost/3929.jpg  
  inflating: /content/Data/weather_dataset/weather-dataset/dataset/frost/3930.jpg  
  inflating: /content/Data/weather_dataset/weather-dataset/dataset/frost/3931.jpg  
  inflating: /content/Data/weather_dataset/weather-dataset/dataset/frost/3932.jpg  
  inflating: /content/Dat

In [5]:
import torch
import torch.nn as nn
import os
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split


In [6]:
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seed = 59
set_seed(seed)


In [8]:
root_dir = '/content/Data/scenes_classification/scenes_classification'
train_dir = os.path.join(root_dir, 'train')
test_dir = os.path.join(root_dir, 'val')

classes = {
    label_idx: class_name \
    for label_idx, class_name in enumerate(
        sorted(os.listdir(train_dir))
    )
}

X_train = []
y_train = []
X_test = []
y_test = []

for dataset_path in [train_dir, test_dir]:
    for label_idx, class_name in classes.items():
        class_dir = os.path.join(dataset_path, class_name)
        for img_filename in os.listdir(class_dir):
            img_path = os.path.join(class_dir, img_filename)
            if 'train' in dataset_path:
                X_train.append(img_path)
                y_train.append(label_idx)
            else:
                X_test.append(img_path)
                y_test.append(label_idx)


In [9]:
seed = 0
val_size = 0.2
is_shuffle = True

X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train,
    test_size=val_size,
    random_state=seed,
    shuffle=is_shuffle
)


In [10]:
class ScenesDataset(Dataset):
    def __init__(
        self,
        X, y,
        transform=None
    ):
        self.transform = transform
        self.img_paths = X
        self.labels = y

    def __len__(self):
        return len(self.img_paths)

    def __getitem__(self, idx):
        img_path = self.img_paths[idx]
        img = Image.open(img_path).convert("RGB")

        if self.transform:
            img = self.transform(img)

        return img, self.labels[idx]


In [11]:
def transform(img, img_size=(224, 224)):
    img = img.resize(img_size)
    img = np.array(img)[..., :3]
    img = torch.tensor(img).permute(2, 0, 1).float()
    normalized_img = img / 255.0

    return normalized_img


In [12]:
train_dataset = ScenesDataset(
    X_train, y_train,
    transform=transform
)
val_dataset = ScenesDataset(
    X_val, y_val,
    transform=transform
)
test_dataset = ScenesDataset(
    X_test, y_test,
    transform=transform
)


In [13]:
train_batch_size = 64
test_batch_size = 8

train_loader = DataLoader(
    train_dataset,
    batch_size=train_batch_size,
    shuffle=True
)
val_loader = DataLoader(
    val_dataset,
    batch_size=test_batch_size,
    shuffle=False
)
test_loader = DataLoader(
    test_dataset,
    batch_size=test_batch_size,
    shuffle=False
)


In [14]:
class BottleneckBlock(nn.Module):
    def __init__(self, in_channels, growth_rate):
        super(BottleneckBlock, self).__init__()
        self.bn1 = nn.BatchNorm2d(in_channels)
        self.conv1 = nn.Conv2d(in_channels, 4 * growth_rate, kernel_size=1, bias=False)
        self.bn2 = nn.BatchNorm2d(4 * growth_rate)
        self.conv2 = nn.Conv2d(4 * growth_rate, growth_rate, kernel_size=3, padding=1, bias=False)
        self.relu = nn.ReLU()

    def forward(self, x):
        res = x.clone().detach()
        x = self.bn1(x)
        x = self.relu(x)
        x = self.conv1(x)
        x = self.bn2(x)
        x = self.relu(x)
        x = self.conv2(x)
        x = torch.cat([res, x], 1)
        return x


class DenseBlock(nn.Module):
    def __init__(self, num_layers, in_channels, growth_rate):
        super(DenseBlock, self).__init__()
        layers = []
        for i in range(num_layers):
            layers.append(BottleneckBlock(in_channels + i * growth_rate, growth_rate))
        self.block = nn.Sequential(*layers)

    def forward(self, x):
        return self.block(x)


In [15]:
class DenseNet(nn.Module):
    def __init__(self, num_blocks, growth_rate, num_classes):
        super(DenseNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 2 * growth_rate, kernel_size=7, padding=3, stride=2, bias=False)
        self.bn1 = nn.BatchNorm2d(2 * growth_rate)
        self.pool1 = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        self.dense_blocks = nn.ModuleList()
        in_channels = 2 * growth_rate

        for i, num_layers in enumerate(num_blocks):
            self.dense_blocks.append(DenseBlock(num_layers, in_channels, growth_rate))
            in_channels += num_layers * growth_rate
            if i != len(num_blocks) - 1:
                out_channels = in_channels // 2
                self.dense_blocks.append(nn.Sequential(
                    nn.BatchNorm2d(in_channels),
                    nn.Conv2d(in_channels, out_channels, kernel_size=1, bias=False),
                    nn.AvgPool2d(kernel_size=2, stride=2)
                ))
                in_channels = out_channels

        self.bn2 = nn.BatchNorm2d(in_channels)
        self.pool2 = nn.AdaptiveAvgPool2d(1)
        self.relu = nn.ReLU()
        self.fc = nn.Linear(in_channels, num_classes)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.pool1(x)

        for block in self.dense_blocks:
            x = block(x)

        x = self.bn2(x)
        x = self.relu(x)
        x = self.pool2(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x


In [16]:
n_classes = len(list(classes.keys()))
device = 'cuda' if torch.cuda.is_available() else 'cpu'

model = DenseNet(
    num_blocks=[6, 12, 24, 16],
    growth_rate=32,
    num_classes=n_classes
).to(device)


In [17]:
lr = 1e-2  # Learning rate
epochs = 25  # Số epochs

criterion = nn.CrossEntropyLoss()  # Hàm loss cho bài toán phân loại
optimizer = torch.optim.SGD(
    model.parameters(),  # Truyền vào tham số của mô hình
    lr=lr  # Learning rate
)


In [19]:
def fit(
    model,
    train_loader,
    val_loader,
    criterion,
    optimizer,
    device,
    epochs
):
    train_losses = []
    val_losses = []

    for epoch in range(epochs):
        batch_train_losses = []

        model.train()  # Đặt mô hình ở chế độ huấn luyện
        for idx, (inputs, labels) in enumerate(train_loader):
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()  # Reset gradient
            outputs = model(inputs)  # Truyền dữ liệu qua mô hình
            loss = criterion(outputs, labels)  # Tính loss
            loss.backward()  # Backpropagation
            optimizer.step()  # Cập nhật trọng số

            batch_train_losses.append(loss.item())

        # Tính trung bình loss cho epoch
        train_loss = sum(batch_train_losses) / len(batch_train_losses)
        train_losses.append(train_loss)

        # Đánh giá trên tập validation
        val_loss, val_acc = evaluate(model, val_loader, criterion, device)
        val_losses.append(val_loss)

        print(f'EPOCH {epoch + 1}:\tTrain Loss: {train_loss:.4f}\tVal Loss: {val_loss:.4f}')

    return train_losses, val_losses


In [21]:
def evaluate(model, dataloader, criterion, device):
    model.eval()  # Đặt mô hình ở chế độ đánh giá
    correct = 0
    total = 0
    losses = []

    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            losses.append(loss.item())

            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    loss = sum(losses) / len(losses)
    acc = correct / total

    return loss, acc


In [22]:
train_losses, val_losses = fit(
    model,
    train_loader,
    val_loader,
    criterion,
    optimizer,
    device,
    epochs
)


EPOCH 1:	Train Loss: 1.2060	Val Loss: 1.1741
EPOCH 2:	Train Loss: 1.1207	Val Loss: 1.1087
EPOCH 3:	Train Loss: 1.0635	Val Loss: 1.0551
EPOCH 4:	Train Loss: 1.0135	Val Loss: 1.0011
EPOCH 5:	Train Loss: 0.9668	Val Loss: 0.9602
EPOCH 6:	Train Loss: 0.9237	Val Loss: 0.9233
EPOCH 7:	Train Loss: 0.8811	Val Loss: 0.8809
EPOCH 8:	Train Loss: 0.8424	Val Loss: 0.8495
EPOCH 9:	Train Loss: 0.8109	Val Loss: 0.8155
EPOCH 10:	Train Loss: 0.7742	Val Loss: 0.8032
EPOCH 11:	Train Loss: 0.7483	Val Loss: 0.7584
EPOCH 12:	Train Loss: 0.7153	Val Loss: 0.7363
EPOCH 13:	Train Loss: 0.6875	Val Loss: 0.7050
EPOCH 14:	Train Loss: 0.6545	Val Loss: 0.7049
EPOCH 15:	Train Loss: 0.6184	Val Loss: 0.6642
EPOCH 16:	Train Loss: 0.5903	Val Loss: 0.6456
EPOCH 17:	Train Loss: 0.5675	Val Loss: 0.8308
EPOCH 18:	Train Loss: 0.5429	Val Loss: 0.6127
EPOCH 19:	Train Loss: 0.5167	Val Loss: 0.6423
EPOCH 20:	Train Loss: 0.4942	Val Loss: 0.6817
EPOCH 21:	Train Loss: 0.4710	Val Loss: 0.6680
EPOCH 22:	Train Loss: 0.4437	Val Loss: 0.81

In [23]:
val_loss, val_acc = evaluate(
    model,
    val_loader,
    criterion,
    device
)

test_loss, test_acc = evaluate(
    model,
    test_loader,
    criterion,
    device
)

print('Evaluation on val/test dataset')
print(f'Val Loss: {val_loss:.4f}, Val Accuracy: {val_acc:.4f}')
print(f'Test Loss: {test_loss:.4f}, Test Accuracy: {test_acc:.4f}')


Evaluation on val/test dataset
Val Loss: 0.6310, Val Accuracy: 0.7748
Test Loss: 0.6056, Test Accuracy: 0.7820
